In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from ssa import *

from darts import TimeSeries
from darts.models import LinearRegressionModel
from darts.metrics import mape, rmse


In [19]:
data = pd.read_parquet("../data/afrr_price.parquet")

In [20]:
data = data[['wind_offshore_dayahead', 'wind_onshore_dayahead',
       'solar_dayahead','aFRR_DownCapPriceEUR', 'aFRR_UpCapPriceEUR', 'load_forecasts',
       'da_price', 'prod_unbalance']]

In [21]:
ts = data["aFRR_UpCapPriceEUR"]
ssa_ts = mySSA(data["aFRR_UpCapPriceEUR"])
ssa_ts.embed(embedding_dimension=128, suspected_frequency=24, verbose=True)
ssa_ts.decompose(True)
components = [i for i in range(10)]
ts_clean = ssa_ts.view_reconstruction(*[ssa_ts.Xs[i] for i in components], names=components, return_df=True, plot=False)

----------------------------------------
EMBEDDING SUMMARY:
Embedding dimension	:  120
Trajectory dimensions	: (120, 3265)
Complete dimension	: (120, 3265)
Missing dimension     	: (120, 0)
----------------------------------------
DECOMPOSITION SUMMARY:
Rank of trajectory		: 120
Dimension of projection space	: 113
Characteristic of projection	: 0.9997


In [22]:
data["aFRR_UpCapPriceEUR_cl"] = ts_clean.values

In [29]:
features_df = data[['wind_offshore_dayahead', 'wind_onshore_dayahead', 'solar_dayahead', 'load_forecasts',
       'da_price', 'prod_unbalance']]

target_df = data[["aFRR_UpCapPriceEUR_cl"]]

In [157]:
def create_lag_features(y, X=None, input_chunk_length=1, future_covariates_length=0):
    if isinstance(y, pd.Series):
        y = y.to_frame(name='y')
    
    # Create empty result DataFrames with same index as y
    past_lags = pd.DataFrame(index=y.index)
    future_covariates = pd.DataFrame(index=y.index) if future_covariates_length > 0 else None
    
    # Create past lags of target variable - use POSITIVE shift for PAST values
    for lag in range(1, input_chunk_length + 1):
        for col in y.columns:
            past_lags[f'{col}_lag_{lag}'] = y[col].shift(lag)  # Positive shift = past values
            if future_covariates_length > 0:
                future_covariates[f'{col}_lag_{lag}'] = 0
    # Add past lags of covariates if provided
    if X is not None:
        for lag in range(1, input_chunk_length + 1):
            for col in X.columns:
                past_lags[f'{col}_lag_{lag}'] = X[col].shift(lag)  # Positive shift = past values
        
        # Add future covariates if requested
        if future_covariates_length > 0:
            for lag in range(1, future_covariates_length + 1):
                for col in X.columns:
                    future_covariates[f'{col}_lag_{lag}'] = X[col].shift(-lag)  # Negative shift = future values
    
    return past_lags.dropna(), future_covariates.dropna()

In [158]:
past_lags, future_df = create_lag_features(target_df, X=features_df, input_chunk_length=2, future_covariates_length=4)

In [159]:
input_chunk_length = 2

In [160]:

last_values = y_test.iloc[:2].copy()

for lag in range(1, input_chunk_length + 1):
    col_name = f'aFRR_UpCapPriceEUR_cl_lag_{lag}'
    
    future_df.loc[future_df.index[0], col_name] = last_values.iloc[-lag]['aFRR_UpCapPriceEUR_cl']
    
    if len(future_df) > 1:
        if lag > 1:
            future_df.loc[future_df.index[1], col_name] = last_values.iloc[-lag+1]['aFRR_UpCapPriceEUR_cl']

/tmp/ipykernel_9991/2280633457.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '59.82702625652235' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  future_df.loc[future_df.index[0], col_name] = last_values.iloc[-lag]['aFRR_UpCapPriceEUR_cl']
/tmp/ipykernel_9991/2280633457.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '52.56213352603012' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  future_df.loc[future_df.index[0], col_name] = last_values.iloc[-lag]['aFRR_UpCapPriceEUR_cl']


In [162]:
model.predict(future_df.iloc[[0]])

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- da_price_lag_3
- da_price_lag_4
- load_forecasts_lag_3
- load_forecasts_lag_4
- prod_unbalance_lag_3
- ...


In [165]:
from sktime.transformations.series.lag import LagTransformer

# Create lag features
lag_transformer = LagTransformer(lags=list(range(1, input_chunk_length+1)))
lagged_features = lag_transformer.fit_transform(data["wind_offshore_dayahead"])

ValueError: A distribution name is required.

time_utc
2024-10-01 22:00:00+00:00     926.166687
2024-10-01 23:00:00+00:00     914.333313
2024-10-02 00:00:00+00:00     900.666687
2024-10-02 01:00:00+00:00     888.500000
2024-10-02 02:00:00+00:00     882.458313
                                ...     
2025-02-20 18:00:00+00:00     996.708313
2025-02-20 19:00:00+00:00    1000.375000
2025-02-20 20:00:00+00:00    1000.291687
2025-02-20 21:00:00+00:00    1002.041687
2025-02-20 22:00:00+00:00    1000.958313
Name: wind_offshore_dayahead, Length: 3384, dtype: float64